In [ ]:
# Convert name to a list of strings
stations_list = [str(station_name.decode('utf-8')).split(' ')[0] for station_name in his['station_name'].values]
print(stations_list)

def get_station_index_from_name(stations_list, station_name):
    """
    Find the index of a station in the list of station names.
    """
    try:
        return stations_list.index(station_name)
    except ValueError:
        print(f"Station '{station_name}' not found in the list.")
        return None
    
def get_station_name_from_index(stations_list, station_index):
    """
    Get the station name for a specific station index.
    """
    if 0 <= station_index < len(stations_list):
        station_name = stations_list[station_index]
        return station_name
    else:
        print(f"Index {station_index} is out of range. Valid range is 0 to {len(stations_list) - 1}.")
        return None
    
def get_station_id(his, station_index):
    """
    Get the station ID for a specific station index.
    """
    station_id = his['station_id'].isel(stations=station_index).values
    return station_id

def get_time(his):
    """
    Get the timestamps from the history file.
    """
    time = his['time'].values
    return time

def get_time_seconds(his):
    time = get_time(his)
    time_seconds = (his['time'] - np.datetime64(time[0])).astype('timedelta64[s]').astype(int)
    return time_seconds

def get_average_dt(his):
    """
    Calculate the average time step (dt) from the history file.
    """
    time_seconds = get_time_seconds(his)
    dt = np.diff(time_seconds)  # Calculate the time differences
    return np.mean(dt)
    
def get_station_coordinates(his, station_index):
    """
    Get the coordinates of a specific station.
    """
    station_x = his['station_x'].isel(stations=station_index).values
    station_y = his['station_y'].isel(stations=station_index).values
    return station_x, station_y

In [ ]:
def load_his_data(his, station_index):
    """
    Load data for a specific station from the history file.
    """
    # Extract all variables for the selected station
    variables = {}
    for var_name in his.variables:
        if 'stations' in his[var_name].dims:
            variables[var_name] = his[var_name].isel(stations=station_index).values
    return variables

def load_specific_variables(variables):
    """
    Load specific variables from the history file.
    """
    # Extract specific variables
    his_dt = get_average_dt(his)
    his_time = get_time(his)
    his_hm0 = variables.get('point_hm0', None)
    his_tp = variables.get('point_tp', None)
    his_wavdir = variables.get('point_wavdir', None)
    his_dirspr = variables.get('point_dirspr', None)
    his_wind_speed = variables.get('wind_speed', None)
    his_wind_direction = variables.get('wind_direction', None)


    return his_dt , his_time, his_hm0, his_tp, his_wavdir, his_dirspr, his_wind_speed, his_wind_direction

In [ ]:
# Plotting functions
def plot_his_data_per_station(df, stations_list):
    """
    Plot the data for multiple stations and overlay their data on the same plots.
    Include a map showing the selected stations with labels.
    """

    fig = plt.figure(figsize=(12, 24))
    fig.suptitle('Hurrywave Model Results for Selected Stations', fontsize=16)

    # Add a map subplot at the top
    map_ax = fig.add_axes([0.1, 0.8, 0.8, 0.15])  # Adjusted position for the map
    m = Basemap(projection='merc', llcrnrlat=50, urcrnrlat=65, llcrnrlon=-5, urcrnrlon=10, resolution='i', ax=map_ax)
    m.drawcoastlines()
    m.fillcontinents(color='lightgray', lake_color='aqua')
    m.drawmapboundary(fill_color='aqua')

    # Plot station locations on the map
    for station_index in station_indices:
        station_name = get_station_name_from_index(stations_list, station_index)
        station_x, station_y = get_station_coordinates(his, station_index)
        print(station_x, station_y)
        x, y = m(station_x, station_y)
        m.plot(x, y, 'ro', markersize=8)
        plt.text(x, y, f' {station_name}', fontsize=8, color='black')

    # Create subplots for the data below the map
    axs = fig.subplots(6, 1, gridspec_kw={'hspace': 0.5, 'top': 0.75})  # Adjusted top to fit below the map

    for station_index in station_indices:
        variables = load_his_data(his, station_index)
        his_dt, his_time, his_hm0, his_tp, his_wavdir, his_dirspr, his_wind_speed, his_wind_direction = load_specific_variables(variables)
        station_name = get_station_name_from_index(stations_list, station_index)

        # Plot Hm0
        axs[0].plot(his_time, his_hm0, label=f'{station_name}')
        axs[0].set_title('Hm0')
        axs[0].set_xlabel('Time')
        axs[0].set_ylabel('Hm0 (m)')
        axs[0].legend()

        # Plot Tp
        axs[1].plot(his_time, his_tp, label=f'{station_name}')
        axs[1].set_title('Tp')
        axs[1].set_xlabel('Time')
        axs[1].set_ylabel('Tp (s)')
        axs[1].legend()

        # Plot Wavdir
        axs[2].plot(his_time, his_wavdir, label=f'{station_name}')
        axs[2].set_title('Wavdir')
        axs[2].set_xlabel('Time')
        axs[2].set_ylabel('Wavdir (degrees)')
        axs[2].legend()

        # Plot Dirspr
        axs[3].plot(his_time, his_dirspr, label=f'{station_name}')
        axs[3].set_title('Dirsper')
        axs[3].set_xlabel('Time')
        axs[3].set_ylabel('Dirsper (degrees)')
        axs[3].legend()

        # Plot Wind Speed
        if his_wind_speed is not None:
            axs[4].plot(his_time, his_wind_speed, label=f'{station_name}')
            axs[4].set_title('Wind Speed')
            axs[4].set_xlabel('Time')
            axs[4].set_ylabel('Wind Speed (m/s)')
            axs[4].legend()

            # Plot Wind Direction
            if his_wind_direction is not None:
                axs[5].plot(his_time, his_wind_direction, label=f'{station_name}')
                axs[5].set_title('Wind Direction')
                axs[5].set_xlabel('Time')
                axs[5].set_ylabel('Wind Direction (degrees)')
                axs[5].legend()
        else:
            axs[4].axis('off')
            axs[5].axis('off')

    plt.show()

In [ ]:
# Example usage
station_indices = list(range(len(stations_list)))  # List of station indices from 0 to len(stations_list) - 1
plot_his_data_per_station(his, station_indices, stations_list)

In [ ]:
# Plotting function ERA5
def plot_data(data, station_dict, show_labels=True):
    """
    Plot time series and map for multiple stations using lat/lon from station_dict.

    Parameters:
        data: xarray.Dataset
        station_dict: dict of {"station_name": (lat, lon)}
        show_labels: bool, optional
            If True, display labels next to station markers on the map. Default is True.
    """
    fig = plt.figure(figsize=(12, 24))
    fig.suptitle('Wave Model Results for Selected Stations', fontsize=16)

    # Add a map subplot at the top
    map_ax = fig.add_axes([0.1, 0.8, 0.8, 0.15])
    m = Basemap(projection='merc', llcrnrlat=50, urcrnrlat=65,
                llcrnrlon=-5, urcrnrlon=10, resolution='i', ax=map_ax)
    m.drawcoastlines()
    m.fillcontinents(color='lightgray', lake_color='aqua')
    m.drawmapboundary(fill_color='aqua')

    # Create subplots
    axs = fig.subplots(6, 1, gridspec_kw={'hspace': 0.5, 'top': 0.75})

    for station_name, (lat, lon) in station_dict.items():
        # Find nearest grid point index
        lat_idx, lon_idx = era5.get_closest_lat_lon_idx_from_coords(data, lat, lon)

        # Get data
        variables = era5.load_station_data(data, lat_idx, lon_idx)
        data_dt, data_time, data_hm0, data_tp, data_wavdir, data_dirspr, data_wind_speed, data_wind_direction = \
            era5.load_specific_variables(data, variables)

        # Plot location on map
        x, y = m(lon, lat)
        m.plot(x, y, 'ro', markersize=8)
        if show_labels:
            map_ax.text(x, y, f' {station_name}', fontsize=8, color='black')

        # Plot time series
        axs[0].plot(data_time, data_hm0, label=f'{station_name}')
        axs[1].plot(data_time, data_tp, label=f'{station_name}')
        axs[2].plot(data_time, data_wavdir, label=f'{station_name}')
        axs[3].plot(data_time, data_dirspr, label=f'{station_name}')

        if data_wind_speed is not None:
            axs[4].plot(data_time, data_wind_speed, label=f'{station_name}')
        if data_wind_direction is not None:
            axs[5].plot(data_time, data_wind_direction, label=f'{station_name}')

    # Titles and labels
    titles = ['Hm0', 'Tp', 'Wave Direction', 'Directional Spread', 'Wind Speed', 'Wind Direction']
    ylabels = ['Hm0 (m)', 'Tp (s)', 'Direction (°)', 'Spread (rad)', 'Speed (m/s)', 'Direction (°)']
    for ax, title, ylabel in zip(axs, titles, ylabels):
        ax.set_title(title)
        ax.set_xlabel('Time')
        ax.set_ylabel(ylabel)
        ax.legend()

    plt.show()

    plot_data(data, station_dict, show_labels=False)

In [ ]:
def plot_station_data_comparison(
    station_name,
    model_df,
    era5_df=None,
    buoy_df=None,
    model_vars=None,
    benchmarks=["era5", "buoy"],
    map_variable_era5=None,
    map_variable_buoy=None,
    df_statistics=None
):
    """
    Plot time series for selected variables at a single station, comparing model, ERA5, and/or buoy data.
    Includes a map with the station location and a side panel showing statistical metrics.

    Parameters:
        station_name (str): Name of the station to plot.
        model_df (dict): Nested dict model[station][variable] = values.
        era5_df (dict): Same structure as model_df for ERA5 (optional).
        buoy_df (dict): Same structure as model_df for buoy data (optional).
        model_vars (list): Model variable names to include in the plot.
        benchmarks (list): Which benchmarks to include ("era5", "buoy", or both).
        variable_mapping_era5 (dict): Mapping from model vars to ERA5 vars.
        variable_mapping_buoy (dict): Mapping from model vars to buoy vars.
        df_statistics (dict): Nested dict of statistics.
    """
    if model_vars is None:
        model_vars = list(model_df.get(station_name, {}).keys())

    n_vars = len(model_vars)

    fig = plt.figure(figsize=(14, 3 * n_vars + 5))
    fig.suptitle(f'Data Comparison at {station_name}', fontsize=16)

    # Add map with station location
    map_ax = fig.add_axes([0.05, 0.87, 0.6, 0.1])
    m = Basemap(projection='merc', llcrnrlat=50, urcrnrlat=65,
                llcrnrlon=-5, urcrnrlon=10, resolution='i', ax=map_ax)
    m.drawcoastlines()
    m.fillcontinents(color='lightgray', lake_color='aqua')
    m.drawmapboundary(fill_color='aqua')

    lat = float(model_df[station_name]["station_y"])
    lon = float(model_df[station_name]["station_x"])
    x, y = m(lon, lat)
    m.plot(x, y, 'ro', markersize=8)
    map_ax.text(x, y, f' {station_name}', fontsize=10, color='black')

    # Create time series plots
    axs = []
    for i in range(n_vars):
        ax = fig.add_axes([0.05, 0.75 - i * 0.2, 0.6, 0.18])
        axs.append(ax)

    # Create stat table column
    stat_axs = []
    for i in range(n_vars):
        ax = fig.add_axes([0.7, 0.75 - i * 0.2, 0.25, 0.18])
        stat_axs.append(ax)

    time = model_df[station_name]["time"]

    for i, var in enumerate(model_vars):
        ax = axs[i]
        stat_ax = stat_axs[i]
        ax.grid(True)

        # Model
        if var in model_df[station_name]:
            ax.plot(time, model_df[station_name][var], label="Model", color='blue')

        # ERA5
        if "era5" in benchmarks and era5_df:
            era5_var = map_variable_era5.get(var, var) if map_variable_era5 else var
            if era5_var in era5_df.get(station_name, {}):
                ax.plot(time, era5_df[station_name][era5_var], label="ERA5", linestyle='--', color='green')

        # Buoy
        if "buoy" in benchmarks and buoy_df:
            buoy_var = map_variable_buoy.get(var, var) if map_variable_buoy else var
            if buoy_var in buoy_df.get(station_name, {}):
                ax.plot(time, buoy_df[station_name][buoy_var], label="Buoy", linestyle=':', color='orange')

        ax.set_title(var)
        ax.set_ylabel(var)
        ax.legend()

        # Display statistics
        stat_ax.axis('off')
        if df_statistics and station_name in df_statistics and var in df_statistics[station_name]:
            lines = []
            for stat in df_statistics[station_name][var]:
                for source in df_statistics[station_name][var][stat]:
                    val = df_statistics[station_name][var][stat][source]
                    lines.append(f"{stat} ({source}): {val:.3f}")
            stat_ax.text(0, 1, '\n'.join(lines), fontsize=9, va='top')

    axs[-1].set_xlabel("Time")
    plt.show()

In [ ]:
specs = {
    # Domain parameters
    'mmax': 421,                    # Number of grid points in x-direction
    'nmax': 481,                    # Number of grid points in y-direction
    'dx': 0.05,                     # Grid spacing in x-direction [degrees]
    'dy': 0.033333333,              # Grid spacing in y-direction [degrees]
    'x0': -12,                      # X-coordinate of the first grid cell corner (1,1) in projected UTM zone [m]
    'y0': 48,                       # Y-coordinate of the first grid cell corner (1,1) in projected UTM zone [m]
    'rotation': 0.0,                # Grid rotation from x-axis in anti-clockwise direction [degrees]

    # Time parameters
    'dt': 300,                      # Time-step size [s]
    'tref': "20131201 000000",      # Reference date
    'tstart': "20131201 000000",    # Start date
    'tstop': "20131208 000000",     # Stop date
    'tspinup': 1800.0,              # Spin-up duration [s]
    't0out': -999.0,                # Specific time for initial output [s]
    't1out': -999.0,                # Specific time for final output [s] - NOT IN SPECS
    'trstout': -999.0,              # Specific time for restart output - NOT IN DOCS
    'dtout': 0,                     # Time-step for global map output [s] - NOT IN SPECS
    'dtmapout': 1800,               # Time-step for map output [s] - NOT IN DOCS
    'dthisout': 300,                # Time-step for observation points output [s]
    'dtsp2out': 3600.0,             # Time-step for specific observation outputs [s]
    'dtwnd': 1800.0,                # Time-interval for wind update [s]
    'dtrstout': 0.0,                # Time-step for binary restart file output [s]   
    'dtmaxout': 0.0,                # Time-step for max water level global map output [s]

    # Model master parameters
    'inputformat': "bin",           # Input format
    'outputformat': "net",          # Output format
    'quadruplets': True,            # Flag for quadruplets in wave modeling [Boolean]
    'refraction': True,             # Activate refraction [Boolean] - NOT IN SPECS
    'gccorr': True,                 # Gravity current correction flag [Boolean] - NOT IN SPECS
    'spinup_meteo': True,           # Meteorological spin-up flag [Boolean]
    'cglim': 1.0,                   # Limiting factor for model parameters - NOT IN SPECS
    'iterperc': -999.0,             # Iteration convergence percentage - NOT IN SPECS
    'iterthresh': 0.1,              # Iteration convergence threshold - NOT IN SPECS
    'spwmergefrac': 0.5,            # Fraction for merging wave parameters - NOT IN SPECS  
    'redopt': 1,                    # Redundancy option
    'alpha': 1.0,                   # Alpha parameter for model adjustments - NOT IN SPECS

    # Physical parameters
    'rhoa': 1.25,                   # Air density [kg/m³]
    'rhow': 1024.0,                 # Water density [kg/m³]
    'fbed': 0.019,                  # Bed friction coefficient - NOT IN SPECS
    'cdcap': 0.0025,                # Cap on drag coefficient
    'winddrag': "zijlema",          # Wind drag formulation ('zijlema' or 'wu')
    'vmax_zijlema': 50.0,           # Max velocity for Zijlema wind drag - NOT IN SPECS

    # Wave parameters
    'dmx1': 0.2,                    # Initial x-direction dispersion
    'dmx2': 1e-05,                  # Final x-direction dispersion
    'freqmin': 0.04,                # Minimum frequency [Hz]
    'freqmax': 0.5,                 # Maximum frequency [Hz]
    'nsigma': 12,                   # Number of frequency bins
    'ntheta': 36,                   # Number of directional bins
    'gammajsp': 3.3,                # JONSWAP gamma
    'gambr': 0.73,                  # Breaking factor - NOT IN SPECS

    # Coordinate reference system (CRS) parameters
    'crs_name': "WGS 84",           # CRS name
    'crs_type': "geographic",       # CRS type
    'crs_epsg': 4326,               # EPSG code for CRS - NOT IN SPECS
    'crsgeo': 0,                    # Geographic CRS flag - NOT IN DOCS
    'crs_utmzone': 'nil',           # UTM zone (if applicable) - NOT IN SPECS
        
    # Configuration files
    'depfile': "hurrywave.dep",       # Elevation (bathymetry and topography) at grid cell centres above reference level [m]
    'mskfile': "hurrywave.msk",       # Mask for inactive (0), active (1), boundary (2), or outflow (3) grid cells
    # 'indexfile': "hurrywave.ind",     # Indices of active grid cells in the overall grid (not used for ASCII input) # NOT IN SPECS
    # 'sbgfile': "hurrywave.wbl",       # Wave blocking coefficients for wave blocking mode # NOT IN SPECS
    'obsfile': "hurrywave.obs",       # Observation points file for output time-series at specific locations
    # 'rstfile': "hurrywave.rst",       # Restart file with fluxes and velocities, written if dtrstout > 0 or trstout > 0 # NOT IN SPECS

    # Forcing - Waves
    'bndfile': "hurrywave.bnd",       # Water-level time-series for boundary cells (msk=2)
    'bhsfile': "hurrywave.bhs",       # Significant wave height time-series per input location [m]
    'btpfile': "hurrywave.btp",       # Peak wave period time-series per input location [s]
    'bwdfile': "hurrywave.bwd",       # Wave direction time-series per input location [degrees]
    'bdsfile': "hurrywave.bds",       # Directional spreading time-series per input location [degrees]
    # 'bspfile': "hurrywave.bsp",       # Wave spectra time-series per input location [degrees] # NOT IN SPECS

    # Forcing - Wind
    # 'spwfile': "hurrywave.spw",       # Spiderweb file with wind speed, direction, pressure, rainfall [m/s, Pa, mm/hr] # NOT IN SPECS
    # 'netspwfile': "spiderweb.nc",     # Spiderweb netCDF file with meteo data [m/s, Pa, mm/hr] # NOT IN SPECS
    'amufile': "wind.amu",            # Delft3D-meteo ASCII wind x-component [m/s]
    'amvfile': "wind.amv",            # Delft3D-meteo ASCII wind y-component [m/s]
    # 'wndfile': "hurrywave.wnd",       # Uniform wind input file [m/s, nautical degrees] # NOT IN SPECS
    # 'netamuamvfile': "hurrywave_netamuamvfile.nc",  # netCDF meteo input with wind x- and y-components [m/s] # NOT IN SPECS
}

# Wind

In [ ]:
wnd_file = r'C:\Users\User\OneDrive\Documents\Python\PYTHON_MSC_CE\Year_2\Python_Thesis\cht_hurrywave\examples\DanielTest\01_data\wind\f998dcs13001_201312061000.wnd'

def read_swn_wnd(filePath, x0inp, y0inp, rot, mxinp, myinp, dxinp, dyinp, idla=3, missing_value = 99):
    
    xx = dxinp*np.arange(0,mxinp+1)
    yy = dyinp*np.arange(0,myinp+1)

    xinp0, yinp0 = np.meshgrid(xx, yy)
    cosrot = np.cos(rot*np.pi/180)
    sinrot = np.sin(rot*np.pi/180)
    xinp = x0inp + xinp0*cosrot - yinp0*sinrot
    yinp = y0inp + xinp0*sinrot + yinp0*cosrot

    if idla==3:
        fid = open(filePath)
        wnd = []
        while True:
            line = fid.readline()

            if (''==line):
                print('eof', filePath)
                break
            
            wnd.append([float(x) for x in line[:-1].split()])

        #Create array with all values in single array
        
        wnds = []
        for wnd_idx in wnd:
            wnds += wnd_idx

        wnds = np.array(wnds)
        wnd[wnd==missing_value] = np.nan

        idxs_u = (myinp+1) * (mxinp+1)

        # wnd files have structure first all east-ward winds, than all northward winds
        wndu = wnds[:idxs_u]
        wndv = wnds[idxs_u:]

        #reshape

        wndu = wndu.reshape(myinp+1, mxinp+1)
        wndv = wndv.reshape(myinp+1, mxinp+1)
        
        return xinp, yinp, wndu, wndv
    else:
        print('other idla\'s not yet scripted')

import glob

wndu = []; wndv = [];
filez = glob.glob(r'C:\Users\User\OneDrive\Documents\Python\PYTHON_MSC_CE\Year_2\Python_Thesis\cht_hurrywave\examples\DanielTest\01_data\wind\*.wnd')
for ix, wnd_file in enumerate(filez):
    xinp, yinp, wndui, wndvi = read_swn_wnd(wnd_file, x0inp=-12, y0inp=48, rot=0, mxinp=210, myinp=240, dxinp=0.1, dyinp=0.06667)
    
    # Add first wnd files 48 times to account for 2 extra days physical spinup

    extra_days = 2

    if ix == 0:
        for i in range(extra_days*24):
            wndu.append(wndui)
            wndv.append(wndvi)
        print(f"Add first wnd file {extra_days*24} times to account for physical spin-up")
    
    wndu.append(wndui)
    wndv.append(wndvi)


wu = np.stack(wndu,0)
wv = np.stack(wndv,0)
tstartinp = specs['tstart']
tstopinp = specs['tstop']
timeinp = pd.date_range(tstartinp, tstopinp, freq='1H')

#%%
class Dataset():

    def __init__(self, time, x, y, u, v, crs=CRS(4326)):

        self.quantity = []
        self.unit     = None  
        self.crs      = crs
        self.time     = time
        self.x        = x
        self.y        = y
        # self.u        = []
        # self.v        = []
        self.add_wind( u, v)

    class uvwind():

        def __init__(self, u, v):
            self.u        = u
            self.v        = v
            self.name     = 'wind'

    def add_wind(self, u, v):
        self.quantity.append(self.uvwind(u, v))

ds = Dataset(timeinp, xinp[0,:], yinp[:,0], wu, wv)
MeteoGrid.write_to_delft3d(ds, 'wind', parameters=['wind'], path=os.path.join(model_setup))

# wind forcing to input file
hw.input.update({'amufile': 'wind.amu', 'amvfile': 'wind.amv', 'tspinup': 1800.0})


# Era 5 support

In [ ]:
import numpy as np
def print_variables(data):
    """
    Print the variables in the dataset.
    """
    # (OPTIONAL) Print available parameters in datatory-file
    print(f"{'Variable Name':<15} {'Long Name':<65} {'Units':<15} {'Dimensions':<40}")
    for var_name, data_array in data.variables.items():
        dimensions = str(data_array.dims)
        long_name = data_array.attrs.get('long_name', 'N/A')
        units = data_array.attrs.get('units', 'N/A')
        print(f"{var_name:<15}  {long_name:<65} {units:<15} {dimensions:<40}")


def load_station_data(data, lat_idx, lon_idx):
    """
    Load all relevant variables at a known station grid point (lat_idx, lon_idx).
    """
    variables = {}
    for var_name in data.data_vars:
        if ('valid_time' in data[var_name].dims and
            'latitude' in data[var_name].dims and
            'longitude' in data[var_name].dims):
            variables[var_name] = data[var_name][:, lat_idx, lon_idx].values
    return variables

def load_specific_variables(data, variables):
    """
    Extract and return selected wave-related variables from the grid point data.
    """
    data_dt = np.diff(data.valid_time.values).astype('timedelta64[s]').mean().astype(float)
    data_time = data.valid_time.values

    data_hm0 = variables.get('swh', None)             # Significant wave height
    data_tp = variables.get('pp1d', None)             # Peak period
    data_wavdir = variables.get('mwd', None)          # Mean wave direction
    data_dirspr = variables.get('wdw', None)          # Directional spread
    data_wind_speed = variables.get('shww', None)     # Wind wave height
    data_wind_direction = variables.get('mdww', None) # Wind wave direction

    return data_dt, data_time, data_hm0, data_tp, data_wavdir, data_dirspr, data_wind_speed, data_wind_direction

def get_number_of_latitude_values(data):
    """
    Get the number of latitude values in the dataset.
    """
    return len(data.latitude.values) if 'latitude' in data.dims else 0

def get_number_of_longitude_values(data):
    """
    Get the number of longitude values in the dataset.
    """
    return len(data.longitude.values) if 'longitude' in data.dims else 0

def get_latitude_values(data):
    """
    Get the latitude values from the dataset.
    """
    return data.latitude.values if 'latitude' in data.dims else None

def get_longitude_values(data):
    """
    Get the longitude values from the dataset.
    """
    return data.longitude.values if 'longitude' in data.dims else None

def get_lat_lon_values(data, lat_idx, lon_idx):
    """
    Get the latitude and longitude values for a specific grid point.
    """
    lat = data.latitude.values[lat_idx] if 'latitude' in data.dims else None
    lon = data.longitude.values[lon_idx] if 'longitude' in data.dims else None
    return lat, lon

def get_lat_lon_values_from_data(data):
    """
    Get the latitude and longitude values from the dataset.
    """
    lat = data.latitude.values if 'latitude' in data.dims else None
    lon = data.longitude.values if 'longitude' in data.dims else None
    return lat, lon

def get_closest_lat_idx_from_coords(data, lat):
    """
    Get the closest latitude index from the dataset for a given latitude coordinate.
    """
    lat_values = get_latitude_values(data)
    if lat_values is not None:
        return np.abs(lat_values - lat).argmin()
    return None

def get_closest_lon_idx_from_coords(data, lon):
    """
    Get the closest longitude index from the dataset for a given longitude coordinate.
    """
    lon_values = get_longitude_values(data)
    if lon_values is not None:
        return np.abs(lon_values - lon).argmin()
    return None

def get_closest_lat_lon_idx_from_coords(data, lat, lon):
    """
    Get the closest latitude and longitude indices from the dataset for given coordinates.
    """
    lat_idx = get_closest_lat_idx_from_coords(data, lat)
    lon_idx = get_closest_lon_idx_from_coords(data, lon)
    return lat_idx, lon_idx

def get_closest_lat_lon_from_list_of_coords(data, lat, lon):
    """
    Get the closest latitude and longitude values from the dataset for given coordinates.
    """
    lat_idx, lon_idx = get_closest_lat_lon_idx_from_coords(data, lat, lon)
    lat_values = get_latitude_values(data)
    lon_values = get_longitude_values(data)
    
    if lat_idx is not None and lon_idx is not None:
        return lat_values[lat_idx], lon_values[lon_idx]
    return None, None

def add_stations_to_dict(station_dict, data, lat_idx, lon_idx):
    """
    Add station metadata (name and coordinates) to the dictionary.
    """
    lat, lon = get_lat_lon_values(data, lat_idx, lon_idx)
    
    # Determine the station name
    if station_dict:
        last_station_name = max(station_dict.keys(), key=lambda k: int(k.replace('station', '')))
        station_number = int(last_station_name.replace('station', '')) + 1
    else:
        station_number = 1
    
    station_name = f'station{station_number:03d}'
    
    # Add station metadata to the dictionary
    station_dict[station_name] = (lat, lon)
    return station_dict

def add_stations_to_dict_from_data(station_dict, data):
    """
    Add all stations to the dictionary using the dataset.
    """
    for lat_idx in range(get_number_of_latitude_values(data)):
        for lon_idx in range(get_number_of_longitude_values(data)):
            station_dict = add_stations_to_dict(station_dict, data, lat_idx, lon_idx)
    return station_dict


def add_stations_to_dict_from_coords(station_dict, data, lat, lon):
    """
    Add station data to the dictionary using latitude and longitude coordinates.
    """
    lat_idx, lon_idx = get_closest_lat_lon_idx_from_coords(data, lat, lon)
    return add_stations_to_dict(station_dict, data, lat_idx, lon_idx)

def add_stations_to_dict_from_coordata_list(station_dict, data, coordata_list):
    """
    Add multiple stations to the dictionary using a list of latitude and longitude coordinates.
    """
    for lat, lon in coordata_list:
        station_dict = add_stations_to_dict_from_coords(station_dict, data, lat, lon)
    return station_dict

def create_station_dict(data):
    """
    Create a dictionary of stations with their latitude and longitude as keys.
    """
    station_dict = {}
    station_dict = add_stations_to_dict_from_data(station_dict, data)
    return station_dict

def print_station_dict(station_dict):
    """
    Print the station dictionary.
    """
    print(f"Number of stations in the dictionary: {len(station_dict)}")
    for station_name, (lat, lon) in station_dict.items():
        print(f"{station_name}: Latitude: {lat}, Longitude: {lon}")


def filter_station_dict(station_dict, data, lat_list, lon_list):
    """
    Filter the station dictionary based on the stations closest to the inputted latitudes and longitudes.
    """
    filtered_dict = {}
    for lat, lon in zip(lat_list, lon_list):
        closest_lat, closest_lon = get_closest_lat_lon_from_list_of_coords(data, lat, lon)
        for station_name, (station_lat, station_lon) in station_dict.items():
            if station_lat == closest_lat and station_lon == closest_lon:
                filtered_dict[station_name] = (station_lat, station_lon)
                break
    return filtered_dict

def change_station_name(station_dict, old_name, new_name):
    """
    Change the name of a station in the dictionary.
    """
    if old_name in station_dict:
        station_dict[new_name] = station_dict.pop(old_name)
    else:
        print(f"Station {old_name} not found in the dictionary.")
    return station_dict

def change_all_station_names_to_new_names(station_dict, new_name):
    """
    Change all station names in the dictionary to a new list of names.
    """
    if len(new_name) != len(station_dict):
        raise ValueError("The number of new names must match the number of stations in the dictionary.")
    
    old_names = list(station_dict.keys())
    for old_name, new_name in zip(old_names, new_name):
        station_dict[new_name] = station_dict.pop(old_name)
    return station_dict